# Query Recommendation

In [ ]:
import copy
import os

from dataLoad import *
from dataProcessing import *

### Loading User Set and Utility Matrix

In [ ]:
userIDs = loadUserSet()
queryIDs, utilityMatrix, average = loadUtilityMatrix()

### Getting Liked and Disliked Queries
In `userQueryLikedDict` and `userQueryDislikedDict`  the key is the User ID and the value is a list of query IDs that each user liked / disliked.
In `queryUserLikedDict` and `queryUserLikedDict` the key is the Query ID and the value is a list of User IDs who liked / disliked the query.

In [ ]:
(
    userQueryLikedDict,
    userQueryDislikedDict,
    queryUserLikedDict,
    queryUserDislikedDict
) = generateLikedDislikedDictionary(
    userIDs, queryIDs, utilityMatrix, average
)

### Calculating the Similarity of Users and Queries with Jaccard Similarity

In [ ]:
userSimilarity = jaccardSimilarity(
    userQueryLikedDict,
    userQueryDislikedDict,
    userIDs
)
querySimilarity = jaccardSimilarity(
    queryUserLikedDict,
    queryUserDislikedDict,
    queryIDs
)

### Getting the unrated queries for prediction


In [ ]:
queriesToPredict = getQueriesToPredict(utilityMatrix)

### Preparing a function to fill predictions
This function is used to fill the prediction of the queries that could not
be predicted. In this specific example average rating of user will be placed.

In [ ]:
fillWithAvg = lambda user, query, average: average[user]
fillPredictions = lambda user, query: fillWithAvg(user, query, average)

### Make Predictions with Query Based Collaborative Filering

In [ ]:
topNQueries = 4
query_cf_predictions = queryBasedCF(utilityMatrix, queriesToPredict, querySimilarity, topNQueries, fillNotPredictable=fillPredictions)

### Make Predictions with User Based Collaborative Filtering

In [ ]:
topNUsers = 4
user_cf_predictions = userBasedCF(utilityMatrix, queriesToPredict, userSimilarity, topNUsers, fillNotPredictable=fillPredictions)

### Fill the Utility Matrix with Predictions

In [ ]:
user_cf_utility = fillUtilityMatrix(utilityMatrix, user_cf_predictions)
query_cf_utility = fillUtilityMatrix(utilityMatrix, query_cf_predictions)

### Return Top K not Posed Queries for a User
Only the queries that were not posed by the user in the past will be returned.

In [ ]:
userId = '100'
k = 5
topKQueries = topKQueriesNotPosed(query_cf_predictions, userId, k)
for i, (queryId, score) in enumerate(topKQueries):
    print(f"{i + 1}. Query {queryId} with score {score}")

### Return Top K Queries From the Filled Utility Matrix
The top K queries will be returned from the filled utility matrix. Some of them may have been posed by the user in the past.


In [ ]:
userId = '100'
k = 5
topKQueries = topKQueriesFromUtilityMatrix(query_cf_utility, userId, k)
for i, (queryId, score) in enumerate(topKQueries):
    print(f"{i + 1}. Query {queryId} with score {score}")